In [2]:
'''Visualization of the filters of VGG16, via gradient ascent in input space.
This script can run on CPU in a few minutes.
Results example: http://i.imgur.com/4nj4KjN.jpg
'''
from __future__ import print_function

import numpy as np
import time
from keras.applications import vgg16
from keras import backend as K

# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv1'

# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


# build the VGG16 network with ImageNet weights
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())


kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img



58892288/58889256 [==============================] - 37s 1us/step
Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

Current loss value: 99.06297
Current loss value: 148.23848
Current loss value: 222.5218
Current loss value: 327.62802
Current loss value: 421.0495
Current loss value: 517.01794
Current loss value: 599.0838
Current loss value: 680.9899
Current loss value: 755.2617
Current loss value: 829.15393
Current loss value: 900.2656
Current loss value: 967.4869
Current loss value: 1036.1135
Current loss value: 1098.7015
Current loss value: 1160.5496
Current loss value: 1227.175
Current loss value: 1288.4001
Current loss value: 1353.5436
Current loss value: 1415.0969
Current loss value: 1476.5309
Filter 8 processed in 11s
Processing filter 9
Current loss value: 15.082466
Current loss value: 25.788387
Current loss value: 59.73834
Current loss value: 119.42493
Current loss value: 173.77098
Current loss value: 213.99406
Current loss value: 257.9475
Current loss value: 316.613
Current loss value: 367.87448
Current loss value: 436.3832
Current loss value: 488.46887
Current loss value: 538.2522
Current l

Current loss value: 136.24715
Current loss value: 165.98625
Current loss value: 201.50766
Current loss value: 234.2843
Current loss value: 275.8061
Current loss value: 312.2529
Current loss value: 352.6621
Current loss value: 391.1461
Current loss value: 425.37598
Current loss value: 463.85846
Current loss value: 495.42398
Current loss value: 532.0067
Current loss value: 564.98145
Current loss value: 601.7849
Filter 25 processed in 12s
Processing filter 26
Current loss value: 0.0
Filter 26 processed in 0s
Processing filter 27
Current loss value: 1.183176
Current loss value: 6.399273
Current loss value: 11.187296
Current loss value: 24.100739
Current loss value: 34.833477
Current loss value: 60.470146
Current loss value: 101.02719
Current loss value: 149.58969
Current loss value: 212.75005
Current loss value: 273.77124
Current loss value: 338.44278
Current loss value: 405.71936
Current loss value: 489.14417
Current loss value: 568.1928
Current loss value: 646.85913
Current loss value: 7

Current loss value: 1127.2449
Current loss value: 1192.3617
Current loss value: 1249.1831
Filter 40 processed in 12s
Processing filter 41
Current loss value: 0.0
Filter 41 processed in 1s
Processing filter 42
Current loss value: 7.6397834
Current loss value: 83.60954
Current loss value: 198.98824
Current loss value: 301.6201
Current loss value: 391.25598
Current loss value: 457.83392
Current loss value: 543.7223
Current loss value: 609.1814
Current loss value: 684.39374
Current loss value: 750.1156
Current loss value: 827.3559
Current loss value: 883.1172
Current loss value: 943.1726
Current loss value: 1015.5191
Current loss value: 1077.999
Current loss value: 1140.8685
Current loss value: 1198.7534
Current loss value: 1254.3894
Current loss value: 1324.3259
Current loss value: 1373.437
Filter 42 processed in 12s
Processing filter 43
Current loss value: 9.45794
Current loss value: 34.37563
Current loss value: 75.694725
Current loss value: 117.09805
Current loss value: 156.90698
Curren

Current loss value: 243.4765
Current loss value: 305.36157
Current loss value: 378.44302
Current loss value: 444.74844
Current loss value: 511.94077
Current loss value: 586.4321
Current loss value: 651.679
Current loss value: 717.93134
Current loss value: 778.1615
Current loss value: 845.3223
Current loss value: 902.20996
Current loss value: 972.5154
Current loss value: 1030.3337
Current loss value: 1090.8203
Filter 56 processed in 14s
Processing filter 57
Current loss value: 0.0
Filter 57 processed in 1s
Processing filter 58
Current loss value: 25.792587
Current loss value: 63.948486
Current loss value: 127.821785
Current loss value: 202.117
Current loss value: 270.15637
Current loss value: 329.6725
Current loss value: 373.22498
Current loss value: 426.95334
Current loss value: 470.2181
Current loss value: 503.9918
Current loss value: 550.27527
Current loss value: 581.5488
Current loss value: 616.7144
Current loss value: 649.73047
Current loss value: 683.9763
Current loss value: 710.0

...

Current loss value: 1058.8418
Current loss value: 1120.9309
Current loss value: 1179.5574
Filter 128 processed in 12s
Processing filter 129
Current loss value: 0.0
Filter 129 processed in 1s
Processing filter 130
Current loss value: 0.0
Filter 130 processed in 1s
Processing filter 131
Current loss value: 5.5249176
Current loss value: 16.973818
Current loss value: 52.898186
Current loss value: 127.17502
Current loss value: 207.29454
Current loss value: 278.51813
Current loss value: 344.9819
Current loss value: 410.4096
Current loss value: 465.99347
Current loss value: 528.77155
Current loss value: 585.42285
Current loss value: 633.83923
Current loss value: 683.30865
Current loss value: 734.0818
Current loss value: 785.34827
Current loss value: 824.18835
Current loss value: 874.96313
Current loss value: 914.6083
Current loss value: 960.552
Current loss value: 999.3995
Filter 131 processed in 12s
Processing filter 132
Current loss value: 2.7699752
Current loss value: 26.287354
Current los

Current loss value: 130.13264
Current loss value: 171.55566
Current loss value: 215.9288
Current loss value: 257.61273
Current loss value: 298.4317
Current loss value: 336.1355
Current loss value: 376.47787
Current loss value: 418.143
Current loss value: 456.58823
Current loss value: 494.12515
Current loss value: 528.39136
Current loss value: 569.9777
Current loss value: 607.55554
Current loss value: 644.9569
Current loss value: 679.39636
Current loss value: 718.18066
Current loss value: 755.6752
Filter 145 processed in 14s
Processing filter 146
Current loss value: 15.458006
Current loss value: 45.80244
Current loss value: 87.19783
Current loss value: 139.03914
Current loss value: 183.97021
Current loss value: 247.47336
Current loss value: 302.7646
Current loss value: 350.86188
Current loss value: 392.8246
Current loss value: 439.42914
Current loss value: 482.77426
Current loss value: 534.0106
Current loss value: 579.93146
Current loss value: 629.58887
Current loss value: 693.3086
Curr

Current loss value: 439.59238
Current loss value: 462.39972
Current loss value: 497.7404
Current loss value: 524.93555
Current loss value: 551.3363
Current loss value: 578.54553
Filter 159 processed in 12s
Processing filter 160
Current loss value: 1.9798347
Current loss value: 14.222093
Current loss value: 47.783867
Current loss value: 88.92436
Current loss value: 135.7575
Current loss value: 191.23152
Current loss value: 245.19733
Current loss value: 296.3681
Current loss value: 343.52148
Current loss value: 384.64893
Current loss value: 430.15338
Current loss value: 473.5302
Current loss value: 515.1122
Current loss value: 559.2567
Current loss value: 598.53467
Current loss value: 639.5165
Current loss value: 678.1948
Current loss value: 718.7212
Current loss value: 755.204
Current loss value: 792.9797
Filter 160 processed in 12s
Processing filter 161
Current loss value: 4.431131
Current loss value: 25.256542
Current loss value: 72.60826
Current loss value: 130.2998
Current loss valu

Current loss value: 71.70261
Current loss value: 125.28031
Current loss value: 177.09166
Current loss value: 229.95288
Current loss value: 280.14865
Current loss value: 318.00388
Current loss value: 371.05853
Current loss value: 416.70386
Current loss value: 464.16257
Current loss value: 514.9376
Current loss value: 562.90643
Current loss value: 607.001
Current loss value: 647.9875
Current loss value: 697.80493
Current loss value: 743.1277
Current loss value: 791.41254
Current loss value: 833.4046
Current loss value: 880.7106
Filter 175 processed in 12s
Processing filter 176
Current loss value: 0.07260416
Current loss value: 26.899044
Current loss value: 91.512596
Current loss value: 175.5279
Current loss value: 266.7572
Current loss value: 351.98328
Current loss value: 422.36035
Current loss value: 491.75812
Current loss value: 562.15076
Current loss value: 628.55927
Current loss value: 696.26965
Current loss value: 762.86847
Current loss value: 825.4358
Current loss value: 886.0982
C

Current loss value: 249.41519
Current loss value: 289.30542
Current loss value: 327.5161
Current loss value: 365.77557
Current loss value: 402.0382
Current loss value: 441.27667
Current loss value: 475.9967
Current loss value: 509.38696
Current loss value: 541.7773
Current loss value: 574.56696
Current loss value: 605.0121
Current loss value: 642.5398
Filter 191 processed in 12s
Processing filter 192
Current loss value: 7.950249
Current loss value: 15.408963
Current loss value: 28.410877
Current loss value: 60.80519
Current loss value: 104.02933
Current loss value: 146.22034
Current loss value: 182.93597
Current loss value: 219.43411
Current loss value: 256.88614
Current loss value: 298.56366
Current loss value: 341.372
Current loss value: 382.47766
Current loss value: 421.43506
Current loss value: 457.96735
Current loss value: 496.93588
Current loss value: 536.1638
Current loss value: 570.06976
Current loss value: 613.9552
Current loss value: 646.0845
Current loss value: 684.75037
Fil